<a href="https://colab.research.google.com/github/svondracek0/Data-Science-Practicum/blob/main/Vondracek_W10_DSP2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install genomic-benchmarks


     |████████████████████████████████| 2.3 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 35.7 MB/s 
     |████████████████████████████████| 271 kB 69.9 MB/s 
     |████████████████████████████████| 160 kB 57.1 MB/s 
  Created wheel for genomic-benchmarks: filename=genomic_benchmarks-0.0.6-py3-none-any.whl size=17525 sha256=20112eba247269aba2bd0b6914b321e23bb72d48126f4e312c315af67cc4c93a
  Stored in directory: /root/.cache/pip/wheels/a7/0f/e4/f723fa5070923dc495b63e02600faf64bf76dbdf422ebdeb66
Successfully built genomic-benchmarks
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install tensorflow>=2.6.0
!pip install tensorflow-addons
!pip install typing-extensions --upgrade 

     |████████████████████████████████| 1.1 MB 5.0 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.0.1 which is incompatible.


In [23]:
from genomic_benchmarks.data_check import list_datasets
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model
 
list_datasets()


['demo_human_or_worm',
 'human_nontata_promoters',
 'human_enhancers_ensembl',
 'demo_mouse_enhancers',
 'human_enhancers_cohn',
 'demo_coding_vs_intergenomic_seqs']

In [37]:
from genomic_benchmarks.data_check import info
info('demo_human_or_worm', version=0)


Dataset `demo_human_or_worm` has 2 classes: human, worm.

All lenghts of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.


,train,test
human,37500,12500
worm,37500,12500


In [38]:
from genomic_benchmarks.loc2seq import download_dataset
download_dataset("demo_human_or_worm", version=0)

Unzipping...Done.


PosixPath('/root/.genomic_benchmarks/demo_human_or_worm')

In [41]:
from pathlib import Path
import tensorflow as tf

BATCH_SIZE = 32
SEQ_TRAIN_PATH = '/root/.genomic_benchmarks/demo_human_or_worm/train'
CLASSES = ['human', 'worm']

In [42]:
train_set = tf.keras.preprocessing.text_dataset_from_directory(
    directory=SEQ_TRAIN_PATH,
    batch_size=BATCH_SIZE,
    class_names=CLASSES
)

Found 75000 files belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [43]:
list(train_set)[0][0][0]

<tf.Tensor: shape=(), dtype=string, numpy=b'ATATACTAAAATAGTGTTGTGATTTTTTATTTTCAGTTCTACAGTGATAGTCAAAATCCAACAGATGTTGGAACTGAAAACGGTGTTGGTACCGCAAATGTTGTTATGAATTGTGTCAACGGACAATGGGTGAAAGAAGGAATTGTAATAAACGATGTCGAGTGTCAAATCATTACCTGAAGGGTATAAGGTGTGAATAT'>

In [44]:


vectorize_layer.adapt(train_set.map(lambda x, y: x))
#vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()



def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_set.map(vectorize_text)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [45]:


model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])



In [46]:
tf.config.run_functions_eagerly(True)


EPOCHS = 10

history = model.fit(
    train_ds,
    epochs=EPOCHS)



Epoch 1/10
2344/2344 [==============================] - 204s 87ms/step - loss: 0.2934 - binary_accuracy: 0.8835 - f1_score: 0.8633
Epoch 2/10
2344/2344 [==============================] - 194s 83ms/step - loss: 0.1994 - binary_accuracy: 0.9212 - f1_score: 0.9148
Epoch 3/10
2344/2344 [==============================] - 211s 90ms/step - loss: 0.1792 - binary_accuracy: 0.9303 - f1_score: 0.9244
Epoch 4/10
2344/2344 [==============================] - 204s 87ms/step - loss: 0.1698 - binary_accuracy: 0.9330 - f1_score: 0.9285
Epoch 5/10
2344/2344 [==============================] - 201s 86ms/step - loss: 0.1623 - binary_accuracy: 0.9355 - f1_score: 0.9308
Epoch 6/10
2344/2344 [==============================] - 204s 87ms/step - loss: 0.1565 - binary_accuracy: 0.9385 - f1_score: 0.9342
Epoch 7/10
2344/2344 [==============================] - 192s 82ms/step - loss: 0.1524 - binary_accuracy: 0.9397 - f1_score: 0.9357
Epoch 8/10
2344/2344 [==============================] - 190s 81ms/step - loss: 0.14

In [47]:


test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    '/root/.genomic_benchmarks/demo_human_or_worm/test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES,
    shuffle=False)

test_ds = test_dset.map(vectorize_text)



Found 25000 files belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [48]:
model.evaluate(test_ds)

782/782 [==============================] - 29s 37ms/step - loss: 0.1652 - binary_accuracy: 0.9348 - f1_score: 0.9188


[0.16521982848644257, 0.9348400235176086, 0.9188383221626282]